In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test_processed = fill_missing_values(X_test)
print(X_train.shape)
print(X_test_processed.shape)

(1212, 828)
(776, 828)


## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [5]:
X_processed,y_processed = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1199, 828)


## 3. Feature Selection

In [6]:
# pearsonr feature selection
from scipy.stats import pearsonr
from sklearn.decomposition import PCA

def pearsonr_feature_selection(X, y, X_test,degree,weights):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlations = {}
#     print("feature_idx: ", feature_idx.T)
    for idx in feature_idx:
        correlation = 0
        for degree_idx in range(0,len(weights)):
            correlation += abs(weights[degree_idx]*pearsonr(X[:,idx],y**degree[degree_idx])[0])
        correlations[idx] = correlation
    data_correlations = pd.DataFrame(correlations, index=['correlation']).T
    indices_desc = data_correlations['correlation'].abs().sort_values(ascending=False).index
#     print(indices_desc)
    return data_correlations,indices_desc

def pearsonr_feature_selection_correlation(X, X_test,threshold):
    feature_idx =  np.array([i for i in range(0,X.shape[1])])
    correlation_select = np.array([True for i in range(0,X.shape[1])])
    for f_idx in feature_idx:
        if(correlation_select[f_idx]):
            feature_co,indices_desc = pearsonr_feature_selection(X[:,(f_idx+1):], X[:,f_idx], X_test_processed,np.array([1]),np.array([1]))
            reomove_features = feature_co.loc[indices_desc][abs(feature_co['correlation']) >= threshold]
            remove_index = reomove_features.index+(f_idx+1)
            correlation_select[remove_index] = False
    
    return X[:,correlation_select],X_test[:,correlation_select]

In [7]:
data_relation,indices_desc = pearsonr_feature_selection(X_processed, y_processed, X_test_processed,np.array([1,-3,3]),np.array([0.9,0.05,0.05]))

In [8]:
selected_features = data_relation.loc[indices_desc][abs(data_relation['correlation']) >= 0.025]
X_selection = X_processed[:,selected_features.index]
X_test_selection = X_test_processed[:,selected_features.index]
print(X_selection.shape)
print(X_test_selection.shape)

(1199, 469)
(776, 469)


In [9]:
# X, X_test = feature_reduction(X, X_test,750)
X_selection_tree, X_test_selection_tree = select_features(X_selection, y_processed, X_test_selection,feature_num = 105)
print("Traing data shape after selection: {}".format(X_selection_tree.shape))
print("Testing data shape after selection: {}".format(X_test_selection_tree.shape))

Traing data shape after selection: (1199, 105)
Testing data shape after selection: (776, 105)


In [10]:
X_selection_tree_cor, X_test_selection_tree_cor = pearsonr_feature_selection_correlation(X_selection_tree, X_test_selection_tree,0.93)
# X_selection_tree_cor, X_test_selection_tree_cor = X_selection_tree, X_test_selection_tree
print(X_selection_tree_cor.shape)
print(X_test_selection_tree_cor.shape)

(1199, 96)
(776, 96)


## 4. Extra Tree

In [11]:
def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
   
#     params = {
#         'task': 'train',
#         'boosting_type': 'gbdt',
#         'objective': 'regression',
#         'num_leaves': 1800,
#         'learning_rate': 0.025,
#         'max_depth': 11,
#         'n_estimators': 1000,
#         'min_child_weight': 1,
#         'subsample': 0.8,
#         'colsample_bytree': 0.8,
#         'feature_fraction': 0.5,
#         'bagging_fraction': 0.8,
#         'bagging_freq': 5,
#         'verbose': -1,
#         'num_iterations':1000,
#     }
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
        # overfitting
    'lambda_l1':0.0,
    'lambda_l2':0.0,
    # control the model complexity
    'num_leaves': 1700,
    'max_depth': 11,
    # randomly select feature to be used for each round
    'feature_fraction': 0.5,
    # percentage of rows used per tree building iteration
    'subsample': 0.8,
    # Small max_bin causes faster speed and large value improves accuracy
    'max_bin':255,
    # recommended to use smaller learning rate with larger iteration
    'learning_rate': 0.025,
    'num_iterations':600,
    # Start with a lower number of trees to build a baseline and increase it later
    'n_estimators': 1000,
        
    'min_child_weight': 1,
    'colsample_bytree': 0.8,
        
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    }
    
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=100,
                    verbose_eval=True
                   )
    
    y_val_pred = gbm.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    y_pred = gbm.predict(X_test) 
    
    return score, y_pred

def train_k_fold(X,X_test, y, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    y_test_predict = np.zeros(X_test.shape[0])
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        y_test_predict += y_pred
        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    return y_test_predict/fold_num

def train_k_fold_predict(X, y, X_test, fold_num=10):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
        
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
        val_score += score
        if(score > 0.5):
            y_test_predict += y_pred
            cnt += 1
    return val_score/fold_num, y_test_predict/cnt

In [12]:
y_test_pred = train_k_fold(X_selection_tree_cor,X_test_selection_tree_cor, y_processed, fold_num=10) 

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24480
[LightGBM] [Info] Number of data points in the train set: 1079, number of used features: 96
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: featur

[45]	training's l2: 33.3057	training's r2: 0.652514	valid_1's l2: 38.3609	valid_1's r2: 0.548268
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[46]	training's l2: 32.6843	training's r2: 0.658998	valid_1's l2: 38.068	valid_1's r2: 0.551718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[47]	training's l2: 32.0971	training's r2: 0.665123	valid_1's l2: 37.697	valid_1's r2: 0.556087
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[48]	training's l2: 31.5456	training's r2: 0.670877	valid_1's l2: 37.3273	valid_1's r2: 0.560441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[49]	training's l2: 31.009	training's r2: 0.676476	valid_1's l2: 37.1187	valid_1's r2: 0.562897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	training's l2: 30.4783	training's r2: 0.682013	valid_1's l2: 36.9226	valid_1's r2: 0.565206
[LightGBM] [Warning] No further splits with po

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[94]	training's l2: 16.5822	training's r2: 0.826994	valid_1's l2: 29.5331	valid_1's r2: 0.652224
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[95]	training's l2: 16.3889	training's r2: 0.829011	valid_1's l2: 29.5006	valid_1's r2: 0.652606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[96]	training's l2: 16.1856	training's r2: 0.831132	valid_1's l2: 29.4764	valid_1's r2: 0.652891
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[97]	training's l2: 16.01	training's r2: 0.832965	valid_1's l2: 29.334	valid_1's r2: 0.654568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[98]	training's l2: 15.8321	training's r2: 0.834821	valid_1's l2: 29.2537	valid_1's r2: 0.655513
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[99]	training's l2: 15.6661	training's r2: 0.836553	valid_1's l2: 29

[168]	training's l2: 8.65021	training's r2: 0.90975	valid_1's l2: 26.9681	valid_1's r2: 0.682428
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	training's l2: 8.59209	training's r2: 0.910357	valid_1's l2: 26.892	valid_1's r2: 0.683325
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	training's l2: 8.55106	training's r2: 0.910785	valid_1's l2: 26.8522	valid_1's r2: 0.683793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[171]	training's l2: 8.46824	training's r2: 0.911649	valid_1's l2: 26.8451	valid_1's r2: 0.683876
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	training's l2: 8.4104	training's r2: 0.912252	valid_1's l2: 26.8258	valid_1's r2: 0.684104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[173]	training's l2: 8.33865	training's r2: 0.913001	valid_1's l2: 26.7999	valid_1's r2: 0.684409
[LightGBM] [Warning] No further splits w

[222]	training's l2: 6.08088	training's r2: 0.936557	valid_1's l2: 26.2567	valid_1's r2: 0.690806
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[223]	training's l2: 6.04847	training's r2: 0.936895	valid_1's l2: 26.2542	valid_1's r2: 0.690836
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[224]	training's l2: 6.00864	training's r2: 0.93731	valid_1's l2: 26.2419	valid_1's r2: 0.690979
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[225]	training's l2: 5.98131	training's r2: 0.937596	valid_1's l2: 26.207	valid_1's r2: 0.691391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[226]	training's l2: 5.93614	training's r2: 0.938067	valid_1's l2: 26.1894	valid_1's r2: 0.691599
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[227]	training's l2: 5.90532	training's r2: 0.938388	valid_1's l2: 26.1729	valid_1's r2: 0.691792
[LightGBM] [Warning] No further splits 

...

[387]	training's l2: 2.3657	training's r2: 0.975034	valid_0's l2: 45.671	valid_0's r2: 0.525981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[388]	training's l2: 2.3541	training's r2: 0.975157	valid_0's l2: 45.7067	valid_0's r2: 0.525611
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[389]	training's l2: 2.34557	training's r2: 0.975247	valid_0's l2: 45.7396	valid_0's r2: 0.525269
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[390]	training's l2: 2.33517	training's r2: 0.975356	valid_0's l2: 45.7461	valid_0's r2: 0.525202
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[391]	training's l2: 2.32116	training's r2: 0.975504	valid_0's l2: 45.7629	valid_0's r2: 0.525027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[392]	training's l2: 2.30782	training's r2: 0.975645	valid_0's l2: 45.7437	valid_0's r2: 0.525226
[LightGBM] [Warning] No further splits w

[437]	training's l2: 1.94288	training's r2: 0.979496	valid_0's l2: 45.2705	valid_0's r2: 0.530137
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[438]	training's l2: 1.93521	training's r2: 0.979577	valid_0's l2: 45.2695	valid_0's r2: 0.530148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[439]	training's l2: 1.93028	training's r2: 0.979629	valid_0's l2: 45.2876	valid_0's r2: 0.52996
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[440]	training's l2: 1.92297	training's r2: 0.979706	valid_0's l2: 45.3029	valid_0's r2: 0.529801
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[441]	training's l2: 1.91408	training's r2: 0.9798	valid_0's l2: 45.306	valid_0's r2: 0.52977
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[442]	training's l2: 1.90388	training's r2: 0.979908	valid_0's l2: 45.3104	valid_0's r2: 0.529723
[LightGBM] [Warning] No further splits wit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[491]	training's l2: 1.5872	training's r2: 0.98325	valid_0's l2: 45.2331	valid_0's r2: 0.530526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[492]	training's l2: 1.58159	training's r2: 0.983309	valid_0's l2: 45.2536	valid_0's r2: 0.530313
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[493]	training's l2: 1.57419	training's r2: 0.983387	valid_0's l2: 45.263	valid_0's r2: 0.530215
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[494]	training's l2: 1.56704	training's r2: 0.983463	valid_0's l2: 45.277	valid_0's r2: 0.53007
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[495]	training's l2: 1.55877	training's r2: 0.98355	valid_0's l2: 45.2571	valid_0's r2: 0.530276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[496]	training's l2: 1.55252	training's r2: 0.983616	valid_0's l2:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[543]	training's l2: 1.29943	training's r2: 0.986287	valid_0's l2: 45.1758	valid_0's r2: 0.53112
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[544]	training's l2: 1.29729	training's r2: 0.986309	valid_0's l2: 45.1732	valid_0's r2: 0.531147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[545]	training's l2: 1.29251	training's r2: 0.98636	valid_0's l2: 45.1617	valid_0's r2: 0.531267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[546]	training's l2: 1.28789	training's r2: 0.986409	valid_0's l2: 45.1843	valid_0's r2: 0.531033
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[547]	training's l2: 1.28433	training's r2: 0.986446	valid_0's l2: 45.1939	valid_0's r2: 0.530933
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[548]	training's l2: 1.28027	training's r2: 0.986489	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[594]	training's l2: 1.10029	training's r2: 0.988388	valid_0's l2: 45.144	valid_0's r2: 0.53145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[595]	training's l2: 1.09699	training's r2: 0.988423	valid_0's l2: 45.1319	valid_0's r2: 0.531576
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[596]	training's l2: 1.09351	training's r2: 0.98846	valid_0's l2: 45.1281	valid_0's r2: 0.531616
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[597]	training's l2: 1.08609	training's r2: 0.988538	valid_0's l2: 45.1456	valid_0's r2: 0.531434
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[598]	training's l2: 1.082	training's r2: 0.988581	valid_0's l2: 45.1259	valid_0's r2: 0.531638
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[599]	training's l2: 1.07834	training's r2: 0.98862	valid_0's l2:

In [13]:
y_test_pred
final_res = np.vstack((indices_test, y_test_pred)).T

In [14]:
score, y_test_pred = train_k_fold_predict(X_selection_tree_cor, y_processed, X_test_selection_tree_cor)
final_res = np.vstack((indices_test, y_test_pred)).T
print("validation score: ", score)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001471 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24480
[LightGBM] [Info] Number of data points in the train set: 1079, number of used features: 96
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: featur

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	training's l2: 21.8689	training's r2: 0.771837	valid_0's l2: 31.8395	valid_0's r2: 0.625064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	training's l2: 21.6017	training's r2: 0.774625	valid_0's l2: 31.7094	valid_0's r2: 0.626595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	training's l2: 21.3303	training's r2: 0.777456	valid_0's l2: 31.5482	valid_0's r2: 0.628494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	training's l2: 21.0527	training's r2: 0.780352	valid_0's l2: 31.5499	valid_0's r2: 0.628473
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	training's l2: 20.7394	training's r2: 0.783621	valid_0's l2: 31.4972	valid_0's r2: 0.629095
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	training's l2: 20.4601	training's r2: 0.786535	valid_0's l2:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[123]	training's l2: 12.3219	training's r2: 0.871443	valid_0's l2: 28.5527	valid_0's r2: 0.663768
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[124]	training's l2: 12.2464	training's r2: 0.872231	valid_0's l2: 28.463	valid_0's r2: 0.664825
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[125]	training's l2: 12.1425	training's r2: 0.873315	valid_0's l2: 28.4545	valid_0's r2: 0.664924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[126]	training's l2: 12.0452	training's r2: 0.87433	valid_0's l2: 28.3633	valid_0's r2: 0.665998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[127]	training's l2: 11.9618	training's r2: 0.875199	valid_0's l2: 28.3229	valid_0's r2: 0.666474
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[128]	training's l2: 11.8612	training's r2: 0.876249	valid_0's

[171]	training's l2: 8.46824	training's r2: 0.911649	valid_0's l2: 26.8451	valid_0's r2: 0.683876
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	training's l2: 8.4104	training's r2: 0.912252	valid_0's l2: 26.8258	valid_0's r2: 0.684104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[173]	training's l2: 8.33865	training's r2: 0.913001	valid_0's l2: 26.7999	valid_0's r2: 0.684409
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[174]	training's l2: 8.26348	training's r2: 0.913785	valid_0's l2: 26.7518	valid_0's r2: 0.684975
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[175]	training's l2: 8.19864	training's r2: 0.914462	valid_0's l2: 26.7329	valid_0's r2: 0.685198
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[176]	training's l2: 8.13584	training's r2: 0.915117	valid_0's l2: 26.6713	valid_0's r2: 0.685923
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[223]	training's l2: 6.04847	training's r2: 0.936895	valid_0's l2: 26.2542	valid_0's r2: 0.690836
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[224]	training's l2: 6.00864	training's r2: 0.93731	valid_0's l2: 26.2419	valid_0's r2: 0.690979
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[225]	training's l2: 5.98131	training's r2: 0.937596	valid_0's l2: 26.207	valid_0's r2: 0.691391
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[226]	training's l2: 5.93614	training's r2: 0.938067	valid_0's l2: 26.1894	valid_0's r2: 0.691599
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[227]	training's l2: 5.90532	training's r2: 0.938388	valid_0's l2: 26.1729	valid_0's r2: 0.691792
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[228]	training's l2: 5.87001	training's r2: 0.938757	valid_0's

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[318]	training's l2: 3.18549	training's r2: 0.966383	valid_1's l2: 46.2663	valid_1's r2: 0.519803
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[319]	training's l2: 3.17381	training's r2: 0.966506	valid_1's l2: 46.2459	valid_1's r2: 0.520014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[320]	training's l2: 3.15721	training's r2: 0.966681	valid_1's l2: 46.2604	valid_1's r2: 0.519864
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[321]	training's l2: 3.14502	training's r2: 0.96681	valid_1's l2: 46.2621	valid_1's r2: 0.519846
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[322]	training's l2: 3.13069	training's r2: 0.966961	valid_1's l2: 46.2457	valid_1's r2: 0.520016
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[323]	training's l2: 3.11873	training's r2: 0.967087	valid_1'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[389]	training's l2: 2.34557	training's r2: 0.975247	valid_1's l2: 45.7396	valid_1's r2: 0.525269
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[390]	training's l2: 2.33517	training's r2: 0.975356	valid_1's l2: 45.7461	valid_1's r2: 0.525202
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[391]	training's l2: 2.32116	training's r2: 0.975504	valid_1's l2: 45.7629	valid_1's r2: 0.525027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[392]	training's l2: 2.30782	training's r2: 0.975645	valid_1's l2: 45.7437	valid_1's r2: 0.525226
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[393]	training's l2: 2.3009	training's r2: 0.975718	valid_1's l2: 45.7217	valid_1's r2: 0.525454
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[394]	training's l2: 2.28969	training's r2: 0.975836	valid_1'

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[454]	training's l2: 1.82484	training's r2: 0.980742	valid_1's l2: 45.374	valid_1's r2: 0.529063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[455]	training's l2: 1.81564	training's r2: 0.980839	valid_1's l2: 45.3752	valid_1's r2: 0.529051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[456]	training's l2: 1.80804	training's r2: 0.980919	valid_1's l2: 45.3634	valid_1's r2: 0.529173
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[457]	training's l2: 1.80187	training's r2: 0.980984	valid_1's l2: 45.3721	valid_1's r2: 0.529083
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[458]	training's l2: 1.7945	training's r2: 0.981062	valid_1's l2: 45.3527	valid_1's r2: 0.529285
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[459]	training's l2: 1.78658	training's r2: 0.981146	valid_1's

[523]	training's l2: 1.40097	training's r2: 0.985215	valid_1's l2: 45.2315	valid_1's r2: 0.530542
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[524]	training's l2: 1.39725	training's r2: 0.985254	valid_1's l2: 45.2129	valid_1's r2: 0.530736
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[525]	training's l2: 1.39351	training's r2: 0.985294	valid_1's l2: 45.2222	valid_1's r2: 0.530639
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[526]	training's l2: 1.38883	training's r2: 0.985343	valid_1's l2: 45.2257	valid_1's r2: 0.530603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[527]	training's l2: 1.38186	training's r2: 0.985417	valid_1's l2: 45.2464	valid_1's r2: 0.530387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[528]	training's l2: 1.37194	training's r2: 0.985522	valid_1's l2: 45.2739	valid_1's r2: 0.530102
[LightGBM] [Warning] No further split

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[593]	training's l2: 1.1027	training's r2: 0.988363	valid_1's l2: 45.1396	valid_1's r2: 0.531496
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[594]	training's l2: 1.10029	training's r2: 0.988388	valid_1's l2: 45.144	valid_1's r2: 0.53145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[595]	training's l2: 1.09699	training's r2: 0.988423	valid_1's l2: 45.1319	valid_1's r2: 0.531576
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[596]	training's l2: 1.09351	training's r2: 0.98846	valid_1's l2: 45.1281	valid_1's r2: 0.531616
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[597]	training's l2: 1.08609	training's r2: 0.988538	valid_1's l2: 45.1456	valid_1's r2: 0.531434
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[598]	training's l2: 1.082	training's r2: 0.988581	valid_1's l2:

In [15]:
df_res = pd.DataFrame(final_res)
df_res.to_csv("our_result.csv", header = ["id", "y"], index=False)